In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from mlxtend.frequent_patterns import association_rules, apriori
from mlxtend.preprocessing import TransactionEncoder as ts
import base64
from IPython.display import HTML

df = pd.read_csv('churn_data.csv')
display(df.head())
ddff = df
ddff.Bandwidth_GB_Year = ddff.Bandwidth_GB_Year.astype('float')

,CaseOrder,Customer_id,Interaction,UID,City,State,County,Zip,Lat,Lng,...,MonthlyCharge,Bandwidth_GB_Year,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8
0,1,K409198,aa90260b-4141-4a24-8e36-b04ce1f4f77b,e885b299883d4f9fb18e39c75155d990,Point Baker,AK,Prince of Wales-Hyder,99927,56.25100,-133.37571,...,172.455519,904.536110,5,5,5,3,4,4,3,4
1,2,S120509,fb76459f-c047-4a9d-8af9-e0f7d4ac2524,f2de8bef964785f41a2959829830fb8a,West Branch,MI,Ogemaw,48661,44.32893,-84.24080,...,242.632554,800.982766,3,4,3,3,4,3,4,4
2,3,K191035,344d114c-3736-4be5-98f7-c72c281e2d35,f1784cfa9f6d92ae816197eb175d3c71,Yamhill,OR,Yamhill,97148,45.35589,-123.24657,...,159.947583,2054.706961,4,4,2,4,4,3,3,3
3,4,D90850,abfa2b40-2d43-4994-b15a-989b8c79e311,dc8a365077241bb5cd5ccd305136b05e,Del Mar,CA,San Diego,92014,32.96687,-117.24798,...,119.956840,2164.579412,4,4,4,2,5,4,3,3
4,5,K662701,68a861fd-0d20-4e51-a587-8a90407ee574,aabb64a116e83fdc4befc1fbab1663f9,Needville,TX,Fort Bend,77461,29.38012,-95.80673,...,149.948316,271.493436,4,4,4,3,4,4,4,5


In [2]:
# Transform Bandwidth to Quartiles for perceivable groups within the market basket analysis
ddff['Bandwidth'] = 0
ddff.loc[(ddff['Bandwidth_GB_Year'] <=1236.47), 'Bandwidth'] = 1
ddff.loc[(ddff['Bandwidth_GB_Year'].between(1236.48, 3279.53)), 'Bandwidth'] = 2
ddff.loc[(ddff['Bandwidth_GB_Year'].between(3279.54, 5586.14)), 'Bandwidth'] = 3
ddff.loc[(ddff['Bandwidth_GB_Year'] >=5586.14), 'Bandwidth'] = 4

# Transform Tenure to Quartiles for perceivable groups within the market basket analysis
ddff['Tenure_T'] = 0
ddff.loc[(ddff['Tenure'] <=7.91), 'Tenure_T'] = 1
ddff.loc[(ddff['Tenure'].between(7.92, 35.43)), 'Tenure_T'] = 2
ddff.loc[(ddff['Tenure'].between(35.44, 61.48)), 'Tenure_T'] = 3
ddff.loc[(ddff['Tenure'] >=61.48), 'Tenure_T'] = 4

# Transform MonthlyCHarge to Quartiles for perceivable groups within the market basket analysis
ddff['Monthly'] = 0
ddff.loc[(ddff['MonthlyCharge'] <=139.98), 'Monthly'] = 1
ddff.loc[(ddff['MonthlyCharge'].between(139.99, 167.48)), 'Monthly'] = 2
ddff.loc[(ddff['MonthlyCharge'].between(167.48, 200.73)), 'Monthly'] = 3
ddff.loc[(ddff['MonthlyCharge'] >=200.74), 'Monthly'] = 4

ddff = ddff[['Churn','Bandwidth','Tenure_T','Monthly','CaseOrder']]

In [3]:
basket = (ddff[ddff['Churn']=='Yes'].groupby(['CaseOrder', 'Tenure_T','Bandwidth','Monthly'])['CaseOrder'].sum().unstack().reset_index().fillna(0).set_index('CaseOrder'))

basket

Monthly,Tenure_T,Bandwidth,0,1,2,3,4
CaseOrder,,,,,,,
2,1,1,0.0,0.0,0.0,0.0,2.0
5,1,1,0.0,0.0,5.0,0.0,0.0
7,2,2,0.0,0.0,0.0,7.0,0.0
8,1,1,0.0,8.0,0.0,0.0,0.0
15,2,2,0.0,0.0,15.0,0.0,0.0
...,...,...,...,...,...,...,...
9933,3,4,0.0,0.0,0.0,0.0,9933.0
9936,4,4,0.0,0.0,0.0,0.0,9936.0
9939,3,4,0.0,0.0,0.0,0.0,9939.0


In [4]:
def one_hot_encoding(x):
    if x<=0:
        return 0
    if x>=1:
        return 1
baskets = basket.applymap(one_hot_encoding)
baskets

Monthly,Tenure_T,Bandwidth,0,1,2,3,4
CaseOrder,,,,,,,
2,1,1,0,0,0,0,1
5,1,1,0,0,1,0,0
7,1,1,0,0,0,1,0
8,1,1,0,1,0,0,0
15,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...
9933,1,1,0,0,0,0,1
9936,1,1,0,0,0,0,1
9939,1,1,0,0,0,0,1


In [5]:
frequencies_of_itemsets =  apriori(baskets, min_support=(.07), use_colnames=True)
rules = association_rules(frequencies_of_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Tenure_T),(Bandwidth),0.999245,1.000000,0.999245,1.000000,1.000000,0.000000,inf
1,(Bandwidth),(Tenure_T),1.000000,0.999245,0.999245,0.999245,1.000000,0.000000,1.000000
2,(Tenure_T),(1),0.999245,0.087170,0.087170,0.087236,1.000755,0.000066,1.000072
3,(1),(Tenure_T),0.087170,0.999245,0.087170,1.000000,1.000755,0.000066,inf
4,(Tenure_T),(4),0.999245,0.480000,0.480000,0.480363,1.000755,0.000362,1.000698
5,(4),(Tenure_T),0.480000,0.999245,0.480000,1.000000,1.000755,0.000362,inf
6,(1),(Bandwidth),0.087170,1.000000,0.087170,1.000000,1.000000,0.000000,inf
7,(Bandwidth),(1),1.000000,0.087170,0.087170,0.087170,1.000000,0.000000,1.000000
8,(Bandwidth),(2),1.000000,0.155849,0.155849,0.155849,1.000000,0.000000,1.000000
9,(2),(Bandwidth),0.155849,1.000000,0.155849,1.000000,1.000000,0.000000,inf


In [6]:
rules.confidence.describe()

count    30.000000
mean      0.656307
std       0.392273
min       0.087170
25%       0.270660
50%       0.999623
75%       1.000000
max       1.000000
Name: confidence, dtype: float64

In [7]:
rules[ (rules['lift']>1) & (rules['confidence']>=.7) & (rules['antecedent support']>.3)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(4),(Tenure_T),0.48,0.999245,0.48,1.0,1.000755,0.000362,inf
26,"(Bandwidth, 4)",(Tenure_T),0.48,0.999245,0.48,1.0,1.000755,0.000362,inf
29,(4),"(Tenure_T, Bandwidth)",0.48,0.999245,0.48,1.0,1.000755,0.000362,inf


In [8]:
ddff.to_csv('fully_prepared_data.csv', sep=',')
def create_download_link(ddff, title = "Download CSV file", filename = "fully_prepared_data.csv"):
    csv = ddff.to_csv(index = False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(ddff)